In [37]:

import urllib.request
import pandas as pd
import requests
import io
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

hsb2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\Vaishnavi\\Auto\\DriverData1.csv")

hsb2["Gap"] = hsb2["Gap"].astype('int64')
hsb2["RelativeSpeed"] = hsb2["RelativeSpeed"].astype('int64')
hsb2["TW"] = hsb2["TW"].astype('category')
hsb2['Choice']=hsb2['Choice']



In [38]:
df=hsb2




# Repeat each row 3 times (including the original)
repeated_df = df.loc[df.index.repeat(3)].reset_index(drop=True)

# Add the new column 'D' with 0s
repeated_df['Observed_Choice'] = 0

# Set the value to 1 at the specified indices
for original_index in df.index:
    repeated_indices = repeated_df.index[original_index*3 : original_index*3 + 3]
    repeated_df.loc[repeated_indices[df.loc[original_index, 'Choice']-1], 'Observed_Choice'] = 1

# Print the modified DataFrame
print("\nModified DataFrame:")
print(repeated_df)



Modified DataFrame:
        ID  SubVehID  Choice  Gap  RelativeSpeed TW  Observed_Choice
0        1         9       2   12              0  0                0
1        1         9       2   12              0  0                1
2        1         9       2   12              0  0                0
3        2        13       1   16              2  1                1
4        2        13       1   16              2  1                0
...    ...       ...     ...  ...            ... ..              ...
7195  2399      3002       3   15              3  1                0
7196  2399      3002       3   15              3  1                1
7197  2400      3002       2   15              4  1                0
7198  2400      3002       2   15              4  1                1
7199  2400      3002       2   15              4  1                0

[7200 rows x 7 columns]


In [46]:
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
theta = np.random.randn(num_features, num_classes)
print(theta)
a = np.exp(np.dot(X, theta))  # Compute 'a' using the dot product

repeated_df['exp'] = a  # Assign 'a' directly as 'exp' column
# Calculate softmax grouped by 'ID'
softmax_by_category = repeated_df.groupby('ID')['exp'].transform(lambda x: np.exp(x) / np.sum(np.exp(x)))

# Assign 'a' and 'softmax' columns to repeated_df

repeated_df['softmax'] = softmax_by_category  # Assign calculated softmax values

print(repeated_df.head())  # Print first few rows to verify

[[ 1.27643114 -0.61920665 -2.72269309]
 [ 2.1531517  -0.40006169 -0.67794147]
 [ 0.05752646 -0.7151739  -1.31920894]]


ValueError: Expected a 1D array, got an array with shape (7200, 3)

In [ ]:
import numpy as np

theta = np.array([
    [ 1.27643114, -0.61920665, -2.72269309],
    [ 2.1531517 , -0.40006169, -0.67794147],
    [ 0.05752646, -0.7151739 , -1.31920894]
])

X = np.array([
    [10, 0, 0],
    [10, 0, 0],
    [10, 0, 0],
    [1, 2, 1],
    [1, 2, 1]
])
[[ 12.7643114   -6.1920665   -2.72269309]
 [ 12.7643114   -6.1920665   -2.72269309]
 [ 12.7643114   -6.1920665   -2.72269309]
 [  5.4871093   -2.85151224  -4.7198435 ]
 [  5.4871093   -2.85151224  -4.7198435 ]]


In [47]:
print(X)

      Gap  RelativeSpeed TW
0      12              0  0
1      12              0  0
2      12              0  0
3      16              2  1
4      16              2  1
...   ...            ... ..
7195   15              3  1
7196   15              3  1
7197   15              4  1
7198   15              4  1
7199   15              4  1

[7200 rows x 3 columns]


In [15]:
repeated_df.to_csv("df.csv", index=False)

In [16]:
hsb3 = repeated_df
hsb3.head()

,ID,SubVehID,Choice,Gap,RelativeSpeed,TW,Observed_Choice
0,1,9,2,12,0,0,0
1,1,9,2,12,0,0,1
2,1,9,2,12,0,0,0
3,2,13,1,16,2,1,1
4,2,13,1,16,2,1,0


In [18]:
y = hsb3['Observed_Choice']
X = hsb3.drop(['ID','SubVehID','Choice','Observed_Choice'],axis=1)

In [ ]:
def softmax(z):
    """
    Compute the softmax of each row of the input array z.
    
    Parameters:
    z (ndarray): Input array of shape (n, k) where n is the number of samples and k is the number of classes.
    
    Returns:
    ndarray: Output array of the same shape as z where each row is the softmax of the corresponding row of z.
    """
      # Subtract max for numerical stability
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

In [19]:

import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import log_loss
# Create a multinomial logistic regression model


model =LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter=1000000)

# Fit the model to the data
model.fit(X, y)
print("Intercept:",model.intercept_)
print("Coefficients:",model.coef_)



# Get the log-likelihood of the fitted model
y_prob = model.predict_proba(X)
LL_model = -log_loss(y, y_prob, normalize=False)

# Fit the null model (only an intercept)
null_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000, fit_intercept=True)
X_null = np.ones((X.shape[0], 1))  # Only intercept
null_model.fit(X_null, y)

# Get the log-likelihood of the null model
y_null_prob = null_model.predict_proba(X_null)
LL_null = -log_loss(y, y_null_prob, normalize=False)


chi_square_stat = 2 * (LL_model - LL_null)
df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (LL_model / LL_null)
print(" ")
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)


Intercept: [-0.34657607]
Coefficients: [[ 1.84833410e-07 -2.90458183e-08 -4.42208346e-07]]
 
Rho-squared: -8.815170815523743e-14
Chi-square statistic: -8.076312951743603e-10
p-value: 1.0


In [11]:

c = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.ID)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict(row)
    # Summarize the predicted probabilities and append to the list
    c.append(yhat[0])
    

hsb2['Predicted'] = c

In [7]:
X.shape

(2400, 3)

In [20]:
import numpy as np

def log_likelihood(X, y, beta):
    """
    Calculate the log likelihood for multinomial logistic regression.
    
    Args:
        X (numpy.ndarray): Design matrix (shape: N x M), where N is the number of samples and M is the number of features.
        y (numpy.ndarray): Target labels (shape: N x 1), where each label corresponds to a class (0, 1, ..., K-1).
        beta (numpy.ndarray): Model coefficients (shape: M x K), where K is the number of classes.
    
    Returns:
        float: Log likelihood value.
    """
    W = np.dot(X, beta)
    eXB = np.exp(W)
    eXB /= eXB.sum(axis=1)[:, None]
    
    log_likelihood_value = np.sum(y * np.log(eXB))
    return log_likelihood_value

# Example usage:
# Assuming you have X (design matrix), y (target labels), and beta (model coefficients)
# log_likelihood_value = log_likelihood(X, y, beta)
# print(f"Log Likelihood: {log_likelihood_value:.4f}")
# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
beta = np.random.randn(num_features, num_classes)

ll = log_likelihood(X, y, beta)
print("Log Likelihood:", ll)

ValueError: operands could not be broadcast together with shapes (7200,) (7200,3) 

In [21]:

from scipy import stats
from scipy.stats import chi2


def softmax(z):
    """
    Compute the softmax of each row of the input array z.
    
    Parameters:
    z (ndarray): Input array of shape (n, k) where n is the number of samples and k is the number of classes.
    
    Returns:
    ndarray: Output array of the same shape as z where each row is the softmax of the corresponding row of z.
    """
      # Subtract max for numerical stability
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

def log_likelihood_multinomial_logistic(X, y, theta):
    """
    Calculate the log likelihood of the data given a multinomial logistic regression model.
    
    Parameters:
    X (ndarray): Input feature matrix of shape (n, d).
    y (ndarray): True class labels of shape (n,).
    theta (ndarray): Model parameters of shape (d, k).
    
    Returns:
    float: Log likelihood value.
    """
    n, d = X.shape
    k = theta.shape[1]
    
    # Compute the linear combination
    linear_combination = np.dot(X, theta)
    
    # Compute the softmax probabilities
    probabilities = softmax(linear_combination)
    
    # Create a one-hot encoded matrix of the true labels
    y_one_hot = np.zeros((n, k))
    y_one_hot[np.arange(n), y] = 1
    
    # Compute the log likelihood
    log_likelihood = np.sum(y_one_hot * np.log(probabilities))
    
    return log_likelihood

# Example usage


# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
theta = np.random.randn(num_features, num_classes)

X_null = np.ones((X.shape[0], 3))  # Only intercept

ll = log_likelihood_multinomial_logistic(X, y, theta)
print("Log Likelihood:", ll)

null_ll = log_likelihood_multinomial_logistic(X_null, y, theta)
print("Null Log Likelihood:", null_ll)

chi_square_stat = 2 * (ll - null_ll)

df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (ll / null_ll)
print(" ")
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)




Log Likelihood: -21373.226860399085
Null Log Likelihood: -4720.558359658337
 
Rho-squared: -3.5276904196447703
Chi-square statistic: -33305.337001481494
p-value: 1.0


In [22]:

from scipy import stats
from scipy.stats import chi2

def softmax(logits):
    """
    Compute the softmax for a set of logits.

    Parameters:
    logits (np.ndarray): An array of logits with shape (n_samples, n_classes).

    Returns:
    np.ndarray: An array of probabilities with the same shape as logits.
    """
    # Subtract the max logits to prevent overflow issues
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    # Calculate softmax by dividing the exponentiated logits by the sum of exponentiated logits for each sample
    probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    return probabilities

def log_likelihood_multinomial_logistic(X, y, theta):
    """
    Calculate the log likelihood of the data given a multinomial logistic regression model.
    
    Parameters:
    X (ndarray): Input feature matrix of shape (n, d).
    y (ndarray): True class labels of shape (n,).
    theta (ndarray): Model parameters of shape (d, k).
    
    Returns:
    float: Log likelihood value.
    """
    n, d = X.shape
    k = theta.shape[1]
    
    # Compute the linear combination
    linear_combination = np.dot(X, theta)
    
    # Compute the softmax probabilities
    probabilities = softmax(linear_combination)
    
    # Create a one-hot encoded matrix of the true labels
    y_one_hot = np.zeros((n, k))
    y_one_hot[np.arange(n), y] = 1
    
    # Compute the log likelihood
    log_likelihood = np.sum(y_one_hot * np.log(probabilities))
    
    return log_likelihood

# Example usage


# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
theta = np.random.randn(num_features, num_classes)

X_null = np.ones((X.shape[0], 3))  # Only intercept

ll = log_likelihood_multinomial_logistic(X, y, theta)
print("Log Likelihood:", ll)

null_ll = log_likelihood_multinomial_logistic(X_null, y, theta)
print("Null Log Likelihood:", null_ll)

chi_square_stat = 2 * (ll - null_ll)

df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (ll / null_ll)
print(" ")
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)



Log Likelihood: -166121.18823852536
Null Log Likelihood: -10433.913614817387
 
Rho-squared: -14.92127310720818
Chi-square statistic: -311374.5492474159
p-value: 1.0


In [18]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import softmax

# Define the softmax function
def softmax(z):


# Define the log-likelihood function
def log_likelihood(params, X, y, num_classes):
    num_samples, num_features = X.shape
    # Reshape the parameter array into a coefficient matrix
    beta = params.reshape(num_classes, num_features + 1)
    # Add a column of ones for the intercept term
    X_intercept = np.hstack([np.ones((num_samples, 1)), X])
    
    # Compute the linear combination
    logits = np.dot(X_intercept, beta.T)
    # Compute the probabilities using the softmax function
    probabilities = softmax(logits)
    # Compute the log-likelihood
    log_likelihood = np.sum(np.log(probabilities[np.arange(num_samples), y]))
    return -log_likelihood  # Return the negative log-likelihood for minimization

# Define a function to fit the model
def fit_multinomial_logistic_regression(X, y, num_classes, max_iter=1000000):
    num_samples, num_features = X.shape
    # Initialize parameters
    initial_params = np.zeros((num_classes, num_features + 1)).ravel()
    # Optimize the log-likelihood function
    result = minimize(log_likelihood, initial_params, args=(X, y, num_classes),
                      method='L-BFGS-B', options={'maxiter': max_iter})
    # Reshape the result into the coefficient matrix
    beta = result.x.reshape(num_classes, num_features + 1)
    return beta


num_classes = 3

# Subtract 1 from y to ensure class labels start from 0


# Fit the model
beta = fit_multinomial_logistic_regression(X, y, num_classes)


print("Intercept and Coefficients:")
print(beta)




Intercept and Coefficients:
[[ 2.14666262 -0.12960308 -0.20794964  0.15761092]
 [-0.45839896 -0.0203722  -0.07868068  0.05502989]
 [-1.68828305  0.14947839  0.28660041 -0.21266022]]


In [13]:
# Function to predict probabilities
def predict_probabilities(X, beta):
    X_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
    logits = np.dot(X_intercept, beta.T)
    probabilities = softmax(logits)
    return probabilities

# Predict probabilities and class labels
c = []
y_pred = []

for i in range(len(hsb2)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    phat = predict_probabilities(row, beta)
    # Summarize the predicted probabilities and append to the list
    c.append(phat[0])
    # Determine the predicted class label (the one with the highest probability)
    y_pred.append(np.argmax(phat))
hsb2['Predicted_1'] = y_pred

In [14]:
# Fitting the MNLogit model
X1 = sm.add_constant(X)

model = sm.MNLogit(y, X1)
result = model.fit()

# Display the summary of the model
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.863698
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                 2400
Model:                        MNLogit   Df Residuals:                     2392
Method:                           MLE   Df Model:                            6
Date:                Thu, 13 Jun 2024   Pseudo R-squ.:                  0.1224
Time:                        12:00:50   Log-Likelihood:                -2072.9
converged:                       True   LL-Null:                       -2361.8
Covariance Type:            nonrobust   LLR p-value:                1.331e-121
     Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.6055      0.604     -4.311      0.000      -3.790      -1.421
Gap               0.

In [15]:
# Predict probabilities
pred_probs = result.predict(X1)

# Predict the class with the highest probability
y_pred = np.argmax(pred_probs.values, axis=1)

# Add predictions to the dataframe
hsb2['Predicted_Class'] = y_pred



In [16]:
hsb2.to_csv("Results.csv", index=False)

In [30]:
n, d = X.shape
k = theta.shape[1]
    

    
    # Create a one-hot encoded matrix of the true labels
y_one_hot = np.zeros((n, k))
y_one_hot[np.arange(n), y] = 1
print(y_one_hot)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


ID                  int64
SubVehID            int64
Choice              int64
Gap                 int64
RelativeSpeed       int64
TW               category
dtype: object